In [1]:
from dotenv import load_dotenv

In [ ]:
import json

In [2]:
import os

In [3]:
load_dotenv()

True

In [4]:
from openai import OpenAI

In [5]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [99]:
def classification():

    while True:
        topic = input("What do you want us to write about?")
    
        classification = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = [
                {"role": "system", "content": f''' 
                For {topic} identify whether this is a valid topic to write an essay on. 
                If it is then give the response in a JSON response, which consists of:
                Yes: If {topic} is valid along with {topic} in the JSON response
                No: If {topic} is not valid
                '''},
    
                {"role": "user", "content": "AI in medical response"},
                {"role": "assistant", "content": "{\"valid\": \"Yes\",\"essay\": \"AI in medical response\"}"},
    
                {"role": "user", "content": "Some random thing"},
                {"role": "assistant", "content": "{\"valid\": \"No\", \"essay\": \"nan\"}"},
    
                {"role":"user", "content": f"{topic}"}
            ],temperature = 0, seed = 1
        )
    
        response_dic = classification.choices[0].message.content
    
        response = json.loads(response_dic)

        if response['valid'] == "Yes":
            return response['essay']
        else:
            print("This doesn't seem to be an essay topic, please try entering a better topic")

In [100]:
classification()

What do you want us to write about? Gujrat


'Gujrat'

In [59]:
def tone_input():
    while True:
        tone = input("What should be the tone? (Professional - P, Casual - C, Technical - T)")
        
        if tone in ["P", "C", "T"]:
            tone_map = {"P": "Professional", "C": "Casual", "T": "Technical"}
            return tone_map[tone]
        else:
            print("Please enter a valid tone!")


In [62]:
def length_input():
    while True:
        length = input("What should be the length? (Short - S, Long -L, Medium - M)")

        if length in ["S", "L", "M"]:
            length_map = {"S": "Short", "L": "Long", "M": "Medium"}
            return length_map[length]
        else:
            print("Please enter a valid length")

In [61]:
def type_input():
    while True:
        type_input = input("What should be the type of post? (blog post - B/social media -S /email- E)")

        if type_input in ["B", "S", "E"]:
            type_map = {"B": "Blog Post", "S": "Social Media Post", "E": "Email"}
            return type_map[type_input]
        else:
            print("Please enter a valid type")

In [85]:
def json_builder():

    temperature_map = {
        "Professional": 0.7,
        "Casual": 1,
        "Technical": 0.3
    }

    classification_input = classification()
    tone = tone_input()
    type_essay = type_input()
    length = length_input()

    temperature = temperature_map.get(tone, 0.6)

    
    data = {
        "Essay Topic": classification_input,
        "Tone": tone,
        "Length": length,
        "Type": type_essay,
        "Temperature": temperature
    }

    return data

In [70]:
json_builder()

What do you want us to write about? ET Money
What should be the tone? (Professional - P, Casual - C, Technical - T) C
What should be the length? (Short - S, Long -L, Medium - M) M
What should be the type of post? (blog post - B/social media -S /email- E) E


{'Essay Topic': 'ET Money',
 'Tone': 'Casual',
 'Length': 'Medium',
 'Type': 'Email'}

In [140]:
def essay_builder():
    essay_data = json_builder()

    tone_map = ["Professional", "Casual", "Technical"]

    type_map = ["Blog Post", "Social Media Post", "Email"]

    type_specs = {
        "Blog Post": "engaging introduction, clear structure, conclusion with CTA",
        "Social Media Post": "hook in first line, scannable content, hashtags",
        "Email": "subject line, professional greeting, concise body"
    }

    length_specs = {
        "Short": "1 paragraph (Max 200 words)",
        "Long": "3 paragraphs (Max 500-700 words)",
        "Medium": "2 paragraphs (Max 200-500 words)" 
        
    }

    classification = client.chat.completions.create(
        # model = "gpt-3.5-turbo",
        model = "gpt-4o",
        messages = [
            {"role": "system", "content": f''' 

            You are a writer, and you need to write 1 {essay_data['Tone']} on the {essay_data['Essay Topic']}

            In the JSON you will also find the length through {essay_data['Type']}, compare the response
            with the {type_specs} and STRICTLY follow those standard.
            
            Finally, in the JSON you will also find the length through {essay_data['Length']}, compare the response
            with the {length_specs} and STRICTLY adjust for the length of the copy based on the specs. 
            

            Write naturally and engagingly and start directly with the {essay_data['Essay Topic']}.
            
            '''}
        ], temperature = essay_data['Temperature'], seed = 3, max_tokens = 1000
    )
    
    
    print(classification.choices[0].message.content)

In [141]:
essay_builder()

What do you want us to write about? ET Money
What should be the tone? (Professional - P, Casual - C, Technical - T) P
What should be the type of post? (blog post - B/social media -S /email- E) E
What should be the length? (Short - S, Long -L, Medium - M) M


Subject Line: Discover the Future of Personal Finance Management with ET Money

---

Dear [Recipient's Name],

I hope this email finds you well. I'm excited to introduce you to ET Money, a revolutionary platform that's transforming the way individuals manage their personal finances. 

ET Money offers a comprehensive suite of features designed to simplify financial planning and investment management. From tracking expenses to investing in mutual funds, ET Money provides a seamless and user-friendly experience tailored to meet the diverse needs of its users. With its intuitive interface, ET Money enables you to make informed financial decisions effortlessly. 

Whether you're looking to optimize your savings or explore new investment opportunities, ET Money empowers you with the tools and insights needed to achieve your financial goals. Don't miss out on the opportunity to take control of your finances with ET Money.

Warm regards,

[Your Name]

[Your Contact Information]


In [143]:
essay_builder()

What do you want us to write about? ET Money
What should be the tone? (Professional - P, Casual - C, Technical - T) C
What should be the type of post? (blog post - B/social media -S /email- E) S
What should be the length? (Short - S, Long -L, Medium - M) S


ET Money is transforming the way we manage our finances with its user-friendly investment platform. Whether you're a seasoned investor or just starting, ET Money offers an array of tools and insights to make financial planning effortless and efficient. #FinanceMadeEasy #InvestSmart 

The app's straightforward interface ensures you spend less time figuring out how to use it and more time on strategizing your investments. With features like zero paperwork and instant account activation, you can focus on growing your wealth with ease. Plus, real-time investment tracking gives you a crystal-clear view of your portfolio's performance. Embrace financial freedom with ET Money, where the power of informed investing is right at your fingertips. 📈🔍 #FinancialFreedom #WealthManagement


In [146]:
essay_builder()

What do you want us to write about? ET Money
What should be the tone? (Professional - P, Casual - C, Technical - T) T
What should be the type of post? (blog post - B/social media -S /email- E) E
What should be the length? (Short - S, Long -L, Medium - M) S


Subject: Discover the Power of ET Money for Smarter Financial Management

Dear [Recipient's Name],

Managing finances can often feel overwhelming, but ET Money is here to simplify the process and empower you to make smarter financial decisions. This innovative platform offers a comprehensive suite of tools designed to help you track expenses, invest wisely, and achieve your financial goals with ease.

ET Money provides a user-friendly interface that allows you to monitor your spending habits, set budgets, and receive personalized insights. With its advanced investment features, you can explore various mutual funds, track their performance, and make informed investment choices. The platform's tax-saving solutions further enhance your financial planning by optimizing your returns.

Whether you're a seasoned investor or just starting your financial journey, ET Money equips you with the knowledge and resources needed to take control of your finances. Experience the convenience of managing 

In [147]:
essay_builder()

What do you want us to write about? ET Money
What should be the tone? (Professional - P, Casual - C, Technical - T) E


Please enter a valid tone!


What should be the tone? (Professional - P, Casual - C, Technical - T) C
What should be the type of post? (blog post - B/social media -S /email- E) E
What should be the length? (Short - S, Long -L, Medium - M) s


Please enter a valid length


What should be the length? (Short - S, Long -L, Medium - M) S


Subject Line: Discover the Power of ET Money for Seamless Financial Management

Hey there,

Have you ever found yourself overwhelmed with juggling multiple financial tasks? ET Money is here to revolutionize the way you manage your finances effortlessly. This app offers a comprehensive platform that unifies your financial planning, budgeting, investing, and tax-saving efforts all in one place.

With ET Money, you can track your expenses, invest in mutual funds, secure insurance, and generate easy-to-understand insights about your financial health. It's designed to save you time and minimize stress, so you can spend less time managing money and more time enjoying life. Plus, with features like personalized portfolio analysis and expenditure tracking, you're always in control.

If you want a smoother journey towards financial freedom, download ET Money today and start conquering your financial goals with confidence.

Best,
[Your Name]


In [176]:
def essay_builder_1(essay_data_1):
    # essay_data_1 = json_builder()

    tone_map_1 = ["Professional", "Casual", "Technical"]

    type_map_1 = ["Blog Post", "Social Media Post", "Email"]

    type_specs_1 = {
        "Blog Post": "engaging introduction, clear structure, conclusion with CTA",
        "Social Media Post": "hook in first line, scannable content, hashtags",
        "Email": "subject line, professional greeting, concise body"
    }

    length_specs_1 = {
        "Short": "1 paragraph (Max 200 words)",
        "Long": "3 paragraphs (Max 500-700 words)",
        "Medium": "2 paragraphs (Max 200-500 words)" 
        
    }

    classification = client.chat.completions.create(
        # model = "gpt-3.5-turbo",
        model = "gpt-4o",
        messages = [
            {"role": "system", "content": f''' 

            You are a writer, and you need to write 1 {essay_data_1['Tone']} on the {essay_data_1['Essay Topic']}

            Carefully go through the JSON response and do the following in the <scratchpad>:

            <scratchpad>

            1. Carefully analyse {essay_data_1['Essay Topic']}
            2. Gather all relavant information, that might be important pertaining to the topic in points in rough
            3. Look at the {essay_data_1['Tone']} and map it to {type_specs_1} and follow those standards strictly
            4. Based on the {type_specs_1}, re-write all the relavant points in that specific tone as adjusted pointers
            5. Look at the {length_specs_1} and convert relavant pointers into the final written document. 
            6. Delete scratchpad and send out the output
            </scratchpad>
            
            '''}
        ], temperature = essay_data_1['Temperature'], seed = 1, max_tokens = 1000
    )
    
    
    content = classification.choices[0].message.content

    return content

In [168]:
essay_builder_1()

What do you want us to write about? ET Money
What should be the tone? (Professional - P, Casual - C, Technical - T) T
What should be the type of post? (blog post - B/social media -S /email- E) E
What should be the length? (Short - S, Long -L, Medium - M) S


**ET Money: Revolutionizing Personal Finance Management**

In today's fast-paced world, managing personal finances efficiently is crucial. ET Money, a comprehensive financial management app, is here to simplify this process. With its user-friendly interface and robust features, ET Money is designed to help individuals take control of their financial health.

**Key Features of ET Money:**

1. **Expense Tracking:** ET Money offers an intuitive expense tracking feature that categorizes your spending, providing insights into your financial habits. This helps users identify areas where they can cut costs and save more.

2. **Investment Management:** The app allows users to manage their investments seamlessly. With options to invest in mutual funds, track stock market trends, and receive personalized investment advice, ET Money ensures your money works for you.

3. **Insurance Management:** ET Money simplifies insurance management by offering a platform to compare policies, calculate premium

'ET_Money.txt'

In [177]:
def file_saver():

    while True:
        essay_data_1 = json_builder()
        essay = essay_builder_1(essay_data_1)

        print(essay)

        satisfied = input("Are you satisfied with the output? (Yes - Y, No - N)")

        if satisfied == "Y":
            file_name = f"{essay_data_1['Essay Topic'].replace(" ", "_")}.txt"
            with open(file_name, "w", encoding = "utf-8") as f:
                f.write(essay)
                
                return file_name
        elif satisfied == "N":
            print("Okay, lets start again")

        else:
            print("Enter a valid input")

In [178]:
file_saver()

What do you want us to write about? History of AI
What should be the tone? (Professional - P, Casual - C, Technical - T) P
What should be the type of post? (blog post - B/social media -S /email- E) S
What should be the length? (Short - S, Long -L, Medium - M) S


**The History of AI: A Journey Through Time**

Artificial Intelligence (AI) has come a long way since its inception, transforming from a mere concept into a reality that shapes modern society. Here's a brief journey through the milestones of AI:

1. **The Birth of AI (1950s):** The term "Artificial Intelligence" was coined by John McCarthy in 1956 during the Dartmouth Conference. This era also saw the development of the first AI programs, such as the Logic Theorist and the General Problem Solver, which laid the groundwork for future advancements.

2. **The Golden Years (1960s-1970s):** AI research flourished with funding and interest. Notable achievements include ELIZA, an early natural language processing program, and Shakey the Robot, the first mobile robot to reason about its actions.

3. **The AI Winter (1970s-1980s):** Despite early successes, AI faced setbacks due to high expectations and limited computational power, leading to reduced funding and interest—a period now known as t

Are you satisfied with the output? (Yes - Y, No - N) N


Okay, lets start again


What do you want us to write about? History of TMKOC
What should be the tone? (Professional - P, Casual - C, Technical - T) P
What should be the type of post? (blog post - B/social media -S /email- E) B
What should be the length? (Short - S, Long -L, Medium - M) S


**Subject: A Brief History of TMKOC: India's Beloved Sitcom**

**Dear Reader,**

I hope this email finds you well. Today, I am excited to share with you the fascinating history of one of India's most beloved television shows, "Taarak Mehta Ka Ooltah Chashmah" (TMKOC).

**The Beginning:**
TMKOC premiered on July 28, 2008, on SAB TV. The show is based on the column "Duniya Ne Undha Chashma" written by Taarak Mehta for the Gujarati weekly magazine "Chitralekha." The sitcom was created by Asit Kumarr Modi and quickly captured the hearts of viewers with its humorous take on everyday life in the fictional Gokuldham Society.

**The Journey:**
Over the years, TMKOC has become a staple in Indian households, known for its engaging storylines and relatable characters. The show addresses social issues with a comedic twist, making it both entertaining and thought-provoking. The ensemble cast, led by Dilip Joshi as Jethalal and Disha Vakani as Daya, has become iconic in Indian television.

**The Imp

Are you satisfied with the output? (Yes - Y, No - N) Y


'History_of_TMKOC.txt'

In [161]:
f""" 
Tried 2 system prompts, and the COT prompt seems to perform better
Both in terms of the output generated + Following the system instructions
"""

' \nTried 2 system prompts, and the COT prompt seems to perform better\nBoth in terms of the output generated + Following the system instructions\n'